In [2]:
from cdag import Graph, load_data, read_table
import pandas as pd

In [3]:
graph = Graph()
load_data(graph)

In [4]:
user = graph.find_one_node('User', naam='TKAPPEN')
for group in graph.groups_for_user(user):
    print(f"\tgroup: {group}")
#for role in graph.roles_for_user(user):
#    print(f"\trole: {role}")
#    for nb in graph.adjacent(role, 'ismember'):
#        print(f"\t\t{nb}")

TKAPPEN Kappen, T.H. (Teus)
	group: ZH0085 SPEANEU Anesthesioloog (UMC)
	group: ZH0125 SPEPYNU Pijnarts (UMC)


In [5]:
#role = graph.find_one_node('Role', naam='ZH001073')
#for rg in graph.effective_groups_for_user(role):
#print(f"{rg._distance}: {rg}")

In [6]:
wc = pd.read_csv('config_workcontext.csv')
wc.set_index('Id', inplace=True)
ws = pd.read_csv('config_wcsegments.csv')
ws.set_index('ConfiguredWorkContextId', inplace=True)

In [7]:
ws.head()

,SegmentId,Disabled
ConfiguredWorkContextId,,
CS00000005,DT00000001,False
CS00000002,CS000366,False
CS00000003,CS00000009,False
CS00000004,CS00000002,False
CS00000004,CS00000006,False


In [8]:
wcs = wc.join(ws, how='inner')
result = wcs.loc[~wcs['Disabled']].copy()
result.to_csv('workcontext.csv')